In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch

from Generation.models import *
from transformers import (
    GPT2Config,
    GPT2Tokenizer
)
from transformers import AutoConfig

In [300]:
cache_dir='../HULK_new/Saved_models/'


#gen_model_name_or_path='../HULK_new/Counterspeech/Saved_Models/Generator/Create_debate_DialoGPT-medium/'
gen_model_name_or_path='../HULK_new/Counterspeech/Saved_Models/Generator/Reddit_DialoGPT-medium/'
#gen_model_name_or_path='microsoft/DialoGPT-medium'

tokenizer = GPT2Tokenizer.from_pretrained(gen_model_name_or_path, cache_dir=cache_dir)
model = Model_Generation.from_pretrained(gen_model_name_or_path,cache_dir=cache_dir)
#model = model.to(device)
model = model.float()

gedi_model_name_or_path = '../HULK_new/Counterspeech/Saved_Models/Discriminator/Emotion_gedi_gpt2_joy/'
gedi_model = Model_Generation.from_pretrained(gedi_model_name_or_path,cache_dir=cache_dir)
#gedi_model.to(device)

no logit scale or bias initialized for gpt2
Loaded config and bias


In [317]:
params = {
    'sep_token':'<|endoftext|>',
    'max_generation_length': 80,
    'min_generation_length':40,
    'max_input_length':128,
    'unpertubed_count':7,
    'num_beams':1,
    'no_repeat_ngram_size':5,
    'repitition_penalty': 3.5,
    'control_type':'gedi',
    'k':100,
    'p':0.92,
    'filter_p':0.8,
    'target_p':0.8,
    'disc_weight':50,
    'class_bias':0,
    'sample':True,
    'temperature':1.0,
    'early_stopping':True,
    'model_path':'gpt2-medium',
    'dataset_hate':'CONAN',
    'task_name':[('Emotion','joy')],
    'coefficient':[4.5],
    'save_path': './../HULK_new/Counterspeech/Results/',
    'device': 'cuda',
    'batch_size':4,
    'cache_path':'./../HULK_new/Saved_models/',
    'generation_method':'huggingface',
}

In [320]:
prompt = "I hate all niggers <|endoftext|>"

start_len=0
text_ids = tokenizer.encode(prompt)
encoded_prompts=torch.LongTensor(text_ids).unsqueeze(0)
# multi_code = tokenizer.encode(secondary_code)
beam_outputs = model.generate(
            controller_alphas=None,
            controller_list=[gedi_model],
            control_type='gedi',
            positive_class='false',
            negative_class='true',
            tokenizer=tokenizer,
            class_bias=params['class_bias'],
            filter_p=params['filter_p'],
            target_p=params['target_p'],
            disc_weight=params['disc_weight'],
            unpertubed_count=params['unpertubed_count'],
            input_ids=encoded_prompts, 
            pad_token_id         = tokenizer.eos_token_id,
            max_length           = params['max_generation_length']+len(encoded_prompts[0]),
            min_length           = params['min_generation_length']+len(encoded_prompts[0]),
            top_k                = params["k"],
            top_p                = params["p"],
            repetition_penalty   = params["repitition_penalty"],
            temperature          = params["temperature"],
            num_beams            = params['num_beams'], 
            do_sample            = params['sample'],
            no_repeat_ngram_size = params['no_repeat_ngram_size'],  
            early_stopping       = params['early_stopping']
        )
reply = (tokenizer.decode(beam_outputs[0])).split(params['sep_token'])[1]
print("Prompt:",prompt)
print("Reply:",reply)

inside own function
torch.Size([1, 7])
token:  ['using']  token id:  [3500]
token:  ['using']  token id:  [3500]
token:  ['Ġrefrain']  token id:  [25133]
token:  ['Ġrefrain']  token id:  [25133]
token:  ['Ġfrom']  token id:  [422]
token:  ['Ġfrom']  token id:  [422]
token:  ['Ġusing']  token id:  [1262]
token:  ['Ġusing']  token id:  [1262]
token:  ['Ġracist']  token id:  [9811]
token:  ['Ġracist']  token id:  [9811]
token:  ['Ġable']  token id:  [1498]
token:  ['Ġable']  token id:  [1498]
token:  ['ist']  token id:  [396]
token:  ['ist']  token id:  [396]
token:  ['Ġlanguage']  token id:  [3303]
token:  ['Ġlanguage']  token id:  [3303]
token:  ['Ġin']  token id:  [287]
tensor([[ 9562, 29688, 25133,   422,  1262, 30367,  1498,   396,  3303]])
torch.Size([2, 9])
torch.Size([2, 50257])
torch.Size([1, 50257])
token:  ['Ġvital']  token id:  [9204]
token:  ['Ġto']  token id:  [284]
torch.Size([2, 50257])
torch.Size([1, 50257])
token:  ['Ġvital']  token id:  [9204]
token:  ['Ġpeople']  token

token:  ['<|endoftext|>']  token id:  [50256]
torch.Size([2, 50257])
torch.Size([1, 50257])
token:  ['<|endoftext|>']  token id:  [50256]
token:  ['Ġnow']  token id:  [783]
torch.Size([2, 50257])
torch.Size([1, 50257])
token:  ['<|endoftext|>']  token id:  [50256]
token:  ['Ġthank']  token id:  [5875]
torch.Size([2, 50257])
torch.Size([1, 50257])
token:  ['Ġdone']  token id:  [1760]
token:  ['<|endoftext|>']  token id:  [50256]
torch.Size([2, 50257])
torch.Size([1, 50257])
token:  ['<|endoftext|>']  token id:  [50256]
token:  ['Ġperson']  token id:  [1048]
torch.Size([2, 50257])
torch.Size([1, 50257])
token:  ['<|endoftext|>']  token id:  [50256]
token:  ['Ġmother']  token id:  [2802]
torch.Size([2, 50257])
torch.Size([1, 50257])
token:  ['<|endoftext|>']  token id:  [50256]
token:  ['Ġof']  token id:  [286]
torch.Size([2, 50257])
torch.Size([1, 50257])
token:  ['Ġtime']  token id:  [640]
token:  ['<|endoftext|>']  token id:  [50256]
torch.Size([2, 50257])
torch.Size([1, 50257])
token:

In [306]:
tokenizer.eos_token_id

50256

In [14]:
encoded_prompts

tensor([[ 1639,   389,   257,  1928,  2475,   220, 50256]])